In [4]:
!nvidia-smi
import numpy as np
np.__version__

Wed Aug  3 15:57:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.72       Driver Version: 512.72       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   59C    P8     5W /  N/A |    685MiB /  4096MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

'1.20.3'

In [5]:
import  os

print(os.getcwd())#获取当前工作目录路径
print(os.path.abspath('.')) #获取当前工作目录路径
# print os.path.abspath('test.txt') #获取当前目录文件下的工作目录路径
print(os.path.abspath('..')) #获取当前工作的父目录 ！注意是父目录路径
print(os.path.abspath(os.curdir))#获取当前工作目录路径

d:\桌面\学习资料\贷款违约预测
d:\桌面\学习资料\贷款违约预测
d:\桌面\学习资料
d:\桌面\学习资料\贷款违约预测


In [6]:
# coding: utf-8
import multiprocessing
from collections import Counter
import xgboost as xgb
import pandas as pd
import numpy as np
import warnings

from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm import tqdm
from sklearn.model_selection import KFold
import gc
from sklearn import preprocessing
from scipy.stats import entropy
# from imblearn.over_sampling import SMOTE
# from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.metrics import roc_auc_score, roc_curve
import datetime
import time
from itertools import product


In [7]:

nowtime = datetime.date.today()
nowtime = str(nowtime)[-5:]
print(nowtime)
warnings.filterwarnings('ignore')


08-03


In [8]:

##读取数据
def load_dataset(DATA_PATH):
    train_label = pd.read_csv(DATA_PATH + 'train.csv')['isDefault']
    train = pd.read_csv(DATA_PATH + 'train.csv')
    test = pd.read_csv(DATA_PATH + 'testA.csv')
    
    feats = [f for f in train.columns if f not in ['isDefault']]
    # train = train[feats]
    test = test[feats]
    print('train.shape', train.shape)
    print('test.shape', test.shape)

    return train_label, train, test


In [9]:
# 处理时间
def transform_time(x):
    day = int(x.split(' ')[0])
    hour = int(x.split(' ')[2].split('.')[0].split(':')[0])
    minute = int(x.split(' ')[2].split('.')[0].split(':')[1])
    second = int(x.split(' ')[2].split('.')[0].split(':')[2])
    return 86400 * day + 3600 * hour + 60 * minute + second


def transform_day(date1):
    date2 = "2020-01-01"
    date1 = time.strptime(date1, "%Y-%m-%d")
    date2 = time.strptime(date2, "%Y-%m-%d")

    # 根据上面需要计算日期还是日期时间，来确定需要几个数组段。下标0表示年，小标1表示月，依次类推...
    # date1=datetime.datetime(date1[0],date1[1],date1[2],date1[3],date1[4],date1[5])
    # date2=datetime.datetime(date2[0],date2[1],date2[2],date2[3],date2[4],date2[5])
    date1 = datetime.datetime(date1[0], date1[1], date1[2])
    date2 = datetime.datetime(date2[0], date2[1], date2[2])
    # 返回两个变量相差的值，就是相差天数
    # print((date2 - date1).days)  # 将天数转成int型
    return (date2 - date1).days


transform_day('2007-09-01')


4505

In [10]:
#编码方式
def labelEncoder_df(df, features):
    for i in features:
        encoder = preprocessing.LabelEncoder()
        df[i] = encoder.fit_transform(df[i])



class MeanEncoder:
    def __init__(self, categorical_features, n_splits=5, target_type='classification', prior_weight_func=None):
        """
        :param categorical_features: list of str, the name of the categorical columns to encode

        :param n_splits: the number of splits used in mean encoding

        :param target_type: str, 'regression' or 'classification'

        :param prior_weight_func:
        a function that takes in the number of observations, and outputs prior weight
        when a dict is passed, the default exponential decay function will be used:
        k: the number of observations needed for the posterior to be weighted equally as the prior
        f: larger f --> smaller slope
        """

        self.categorical_features = categorical_features
        self.n_splits = n_splits
        self.learned_stats = {}

        if target_type == 'classification':
            self.target_type = target_type
            self.target_values = []
        else:
            self.target_type = 'regression'
            self.target_values = None

        if isinstance(prior_weight_func, dict):
            self.prior_weight_func = eval('lambda x: 1 / (1 + np.exp((x - k) / f))', dict(prior_weight_func, np=np))
        elif callable(prior_weight_func):
            self.prior_weight_func = prior_weight_func
        else:
            self.prior_weight_func = lambda x: 1 / (1 + np.exp((x - 2) / 1))

    @staticmethod
    def mean_encode_subroutine(X_train, y_train, X_test, variable, target, prior_weight_func):
        X_train = X_train[[variable]].copy()
        X_test = X_test[[variable]].copy()

        if target is not None:
            nf_name = '{}_pred_{}'.format(variable, target)
            X_train['pred_temp'] = (y_train == target).astype(int)  # classification
        else:
            nf_name = '{}_pred'.format(variable)
            X_train['pred_temp'] = y_train  # regression
        prior = X_train['pred_temp'].mean()

        col_avg_y = X_train.groupby(by=variable, axis=0)['pred_temp'].agg([('mean', 'mean'), ('beta', 'size'),])
        col_avg_y['beta'] = prior_weight_func(col_avg_y['beta'])
        col_avg_y[nf_name] = col_avg_y['beta'] * prior + (1 - col_avg_y['beta']) * col_avg_y['mean']
        col_avg_y.drop(['beta', 'mean'], axis=1, inplace=True)

        nf_train = X_train.join(col_avg_y, on=variable)[nf_name].values
        nf_test = X_test.join(col_avg_y, on=variable).fillna(prior, inplace=False)[nf_name].values

        return nf_train, nf_test, prior, col_avg_y

    def fit_transform(self, X, y):
        """
        :param X: pandas DataFrame, n_samples * n_features
        :param y: pandas Series or numpy array, n_samples
        :return X_new: the transformed pandas DataFrame containing mean-encoded categorical features
        """
        X_new = X.copy()
        if self.target_type == 'classification':
            skf = StratifiedKFold(self.n_splits)
        else:
            skf = KFold(self.n_splits)

        if self.target_type == 'classification':
            self.target_values = sorted(set(y))
            self.learned_stats = {'{}_pred_{}'.format(variable, target): [] for variable, target in
                                  product(self.categorical_features, self.target_values)}
            for variable, target in product(self.categorical_features, self.target_values):
                nf_name = '{}_pred_{}'.format(variable, target)
                X_new.loc[:, nf_name] = np.nan
                for large_ind, small_ind in skf.split(y, y):
                    nf_large, nf_small, prior, col_avg_y = MeanEncoder.mean_encode_subroutine(
                        X_new.iloc[large_ind], y.iloc[large_ind], X_new.iloc[small_ind], variable, target,
                        self.prior_weight_func)
                    X_new.iloc[small_ind, -1] = nf_small
                    self.learned_stats[nf_name].append((prior, col_avg_y))
        else:
            self.learned_stats = {'{}_pred'.format(variable): [] for variable in self.categorical_features}
            for variable in self.categorical_features:
                nf_name = '{}_pred'.format(variable)
                X_new.loc[:, nf_name] = np.nan
                for large_ind, small_ind in skf.split(y, y):
                    nf_large, nf_small, prior, col_avg_y = MeanEncoder.mean_encode_subroutine(
                        X_new.iloc[large_ind], y.iloc[large_ind], X_new.iloc[small_ind], variable, None,
                        self.prior_weight_func)
                    X_new.iloc[small_ind, -1] = nf_small
                    self.learned_stats[nf_name].append((prior, col_avg_y))
        return X_new

    def transform(self, X):
        """
        :param X: pandas DataFrame, n_samples * n_features
        :return X_new: the transformed pandas DataFrame containing mean-encoded categorical features
        """
        X_new = X.copy()

        if self.target_type == 'classification':
            for variable, target in product(self.categorical_features, self.target_values):
                nf_name = '{}_pred_{}'.format(variable, target)
                X_new[nf_name] = 0
                for prior, col_avg_y in self.learned_stats[nf_name]:
                    X_new[nf_name] += X_new[[variable]].join(col_avg_y, on=variable).fillna(prior, inplace=False)[
                        nf_name]
                X_new[nf_name] /= self.n_splits
        else:
            for variable in self.categorical_features:
                nf_name = '{}_pred'.format(variable)
                X_new[nf_name] = 0
                for prior, col_avg_y in self.learned_stats[nf_name]:
                    X_new[nf_name] += X_new[[variable]].join(col_avg_y, on=variable).fillna(prior, inplace=False)[
                        nf_name]
                X_new[nf_name] /= self.n_splits

        return X_new


In [11]:
#employmentLength, earliesCreditLine_month, gradeTrans, subGradeTrans字段处理

def employmentLength_deal(x):
    #缺失值以-999填充
    if x == r'\N':
        result = -999
    elif x == -999:
        result = -999
    elif x == '-999':
        result = -999
    elif x == '< 1 year':
        result = 0.5
    elif x == '10+ years':
        result = 12
    else:
        result = int(x.split(' ')[0][0])
    # print(result)
    return result


def earliesCreditLine_month_deal(x):
    x = x.split('-')[0]
    # print(x)
    dict = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10,
            'Nov': 11, 'Dec': 12}
    result = dict[x]
    return result


def gradeTrans(x):
    dict = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7}
    result = dict[x]
    return result


def subGradeTrans(x):
    dict = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7}
    result = dict[x[0]]
    result = result * 5 + int(x[1])
    return result


In [12]:
#暴力提取特征的统计量作为新特征

def myEntro(x):
    """
        calculate shanno ent of x
    """
    x = np.array(x)
    x_value_list = set([x[i] for i in range(x.shape[0])])
    ent = 0.0
    for x_value in x_value_list:
        p = float(x[x == x_value].shape[0]) / x.shape[0]
        logp = np.log2(p)
        ent -= p * logp
    #     print(x_value,p,logp)
    # print(ent)
    return ent


def myRms(records):
    records = list(records)
    """
    均方根值 反映的是有效值而不是平均值
    """
    return np.math.sqrt(sum([x ** 2 for x in records]) / len(records))


def myMode(x):
    return np.mean(pd.Series.mode(x))


def myQ25(x):
    return x.quantile(0.25)


def myQ75(x):
    return x.quantile(0.75)


def myRange(x):
    return pd.Series.max(x) - pd.Series.min(x)



In [13]:

# 数据预处理（训练数据与预测数据合并处理）
def data_preprocess(DATA_PATH):
    train_label, train, test = load_dataset(DATA_PATH=DATA_PATH)
    # 拼接数据

    data = pd.concat([train, test], axis=0, ignore_index=True)
    print('初始拼接后：', data.shape)
    # n_feat = [f for f in data.columns if f[0] == 'n']

    n_feat = ['n0', 'n1', 'n2', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'n10', 'n11', 'n12', 'n13', 'n14', ]
    # nameList = ['min', 'max', 'sum', 'mean', 'median', 'skew', 'std', 'mode', 'range']
    # statList = ['min', 'max', 'sum', 'mean', 'median', 'skew', 'std', myMode, myRange]
    nameList = ['max', 'sum', 'mean', 'median', 'skew', 'std']
    statList = ['max', 'sum', 'mean', 'median', 'skew', 'std']


    for i in range(len(nameList)):
        data['n_feat_{}'.format(nameList[i])] = data[n_feat].agg(statList[i], axis=1)
    print('n特征处理后：', data.shape)


    # count编码
    count_list = ['subGrade', 'grade', 'postCode', 'regionCode','homeOwnership','title','employmentTitle','employmentLength']
    data = count_coding(data, count_list)
    print('count编码后：', data.shape)
    ### 用数值特征对类别特征做统计刻画，随便挑了几个跟price相关性最高的匿名特征
    cross_cat = ['subGrade', 'grade', 'employmentLength', 'term', 'homeOwnership', 'postCode', 'regionCode','employmentTitle','title']
    cross_num = ['dti', 'revolBal','revolUtil', 'ficoRangeHigh', 'interestRate', 'loanAmnt', 'installment', 'annualIncome', 'n14',
                 'n2', 'n6', 'n9', 'n5', 'n8']

    data[['employmentLength']].fillna(-999, inplace=True)

#     data = cross_cat_num(data, cross_num, cross_cat)  # 一阶交叉
#     print('一阶特征处理后：', data.shape)
#     data = cross_qua_cat_num(data)  # 二阶交叉
#     print('二阶特征处理后：', data.shape)
    # 缺失值处理
    for temp in count_list:
        del data[temp+'_count']
    # num_fill_col = ['employmentLength', 'postCode', ]
    cols = ['employmentTitle', 'employmentLength', 'postCode', 'dti', 'pubRecBankruptcies', 'revolUtil', 'title',
            'n0', 'n1', 'n2', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'n10', 'n11', 'n12', 'n13', 'n14']
    for col in cols:
        data[col].fillna(r'\N', inplace=True)
    cols = [f for f in cols if f not in ['employmentLength']]
    for col in cols:
        data[col].replace({r'\N': -999}, inplace=True)
        data[col] = data[col]
    # print('缺失值情况：', data.isnull().sum())

    data['grade'] = data['grade'].apply(lambda x: gradeTrans(x))
    data['subGrade'] = data['subGrade'].apply(lambda x: subGradeTrans(x))
    print('1data.shape', data.shape)

    data['employmentLength'] = data['employmentLength'].apply(lambda x: employmentLength_deal(x))
    data['issueDate_year'] = data['issueDate'].apply(lambda x: int(x.split('-')[0]))
    data['issueDate_month'] = data['issueDate'].apply(lambda x: int(x.split('-')[1]))
    data['issueDate_day'] = data['issueDate'].apply(lambda x: transform_day(x))
    data['issueDate_week'] = data['issueDate_day'].apply(lambda x: int(x % 7) + 1)

    print('2_data.shape', data.shape)
    data['earliesCreditLine_year'] = data['earliesCreditLine'].apply(lambda x: 2020 - (int(x.split('-')[-1])))
    data['earliesCreditLine_month'] = data['earliesCreditLine'].apply(lambda x: earliesCreditLine_month_deal(x))
    data['earliesCreditLine_Allmonth'] = data['earliesCreditLine_year'] * 12 - data['earliesCreditLine_month']
    del data['issueDate'], data['earliesCreditLine']

    print('预处理完毕', data.shape)

    return data, train_label


In [14]:
# 常规的 target encoder 目标编码代码容易造成过拟合，因此引入 5 折交叉验证的形式改进编码
# 即将样本分为 5 块（fold），每一 fold 中的该高基数无序特征类别由其它 4 个 fold 中的类别对应标签平均值替换表示：

def kfold_stats_feature(train, test, feats, k):
    folds = StratifiedKFold(n_splits=k, shuffle=True, random_state=6666)  # 这里最好和后面模型的K折交叉验证保持一致

    train['fold'] = None
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, train['isDefault'])):
        train.loc[val_idx, 'fold'] = fold_

    kfold_features = []
    for feat in feats:
        nums_columns = ['isDefault']
        for f in nums_columns:
            colname = feat + '_' + f + '_kfold_mean'
            kfold_features.append(colname)
            train[colname] = None
            for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, train['isDefault'])):
                tmp_trn = train.iloc[trn_idx]
                order_label = tmp_trn.groupby([feat])[f].mean()
                tmp = train.loc[train.fold == fold_, [feat]]
                train.loc[train.fold == fold_, colname] = tmp[feat].map(order_label)
                # fillna
                global_mean = train[f].mean()
                train.loc[train.fold == fold_, colname] = train.loc[train.fold == fold_, colname].fillna(global_mean)
            train[colname] = train[colname].astype(float)

        for f in nums_columns:
            colname = feat + '_' + f + '_kfold_mean'
            test[colname] = None
            order_label = train.groupby([feat])[f].mean()
            test[colname] = test[feat].map(order_label)
            # fillna
            global_mean = train[f].mean()
            test[colname] = test[colname].fillna(global_mean)
            test[colname] = test[colname].astype(float)
    del train['fold']
    return train, test


In [15]:

def GridSearch(clf, params, X, y):
    cscv = GridSearchCV(clf, params, scoring='roc_auc', n_jobs=4, cv=10)
    cscv.fit(X, y)
    print(cscv.cv_results_)
    print(cscv.best_params_)
    print(cscv.best_score_)


In [16]:

# 定义离散型特征和连续型特征交叉特征统计函数
def cross_cat_num(df, num_col, cat_col):
    for f1 in tqdm(cat_col):
        g = df.groupby(f1, as_index=False)
        for f2 in tqdm(num_col):
            feat = g[f2].agg({
                '{}_{}_max'.format(f1, f2): 'max', '{}_{}_min'.format(f1, f2): 'min',
                '{}_{}_median'.format(f1, f2): 'median',
            })
            df = df.merge(feat, on=f1, how='left')
    return (df)

#类别特征的二级交叉
def cross_qua_cat_num(df):
    for f_pair in tqdm([
        ['subGrade', 'regionCode'], ['grade', 'regionCode'], ['subGrade', 'postCode'], ['grade', 'postCode'], ['employmentTitle','title'],
        ['regionCode','title'], ['postCode','title'], ['homeOwnership','title'], ['homeOwnership','employmentTitle'],['homeOwnership','employmentLength'],
        ['regionCode', 'postCode']
    ]):
        ### 共现次数
        df['_'.join(f_pair) + '_count'] = df.groupby(f_pair)['id'].transform('count')
        ### n unique、熵
        df = df.merge(df.groupby(f_pair[0], as_index=False)[f_pair[1]].agg({
            '{}_{}_nunique'.format(f_pair[0], f_pair[1]): 'nunique',
            '{}_{}_ent'.format(f_pair[0], f_pair[1]): lambda x: entropy(x.value_counts() / x.shape[0])
        }), on=f_pair[0], how='left')
        df = df.merge(df.groupby(f_pair[1], as_index=False)[f_pair[0]].agg({
            '{}_{}_nunique'.format(f_pair[1], f_pair[0]): 'nunique',
            '{}_{}_ent'.format(f_pair[1], f_pair[0]): lambda x: entropy(x.value_counts() / x.shape[0])
        }), on=f_pair[1], how='left')
        ### 比例偏好
        df['{}_in_{}_prop'.format(f_pair[0], f_pair[1])] = df['_'.join(f_pair) + '_count'] / df[f_pair[1] + '_count']
        df['{}_in_{}_prop'.format(f_pair[1], f_pair[0])] = df['_'.join(f_pair) + '_count'] / df[f_pair[0] + '_count']
    return (df)



In [17]:

### count编码
def count_coding(df, fea_col):
    for f in fea_col:
        df[f + '_count'] = df[f].map(df[f].value_counts())
    return (df)


In [18]:
#新构造的有实际意义的特征
def gen_basicFea(data):
    data['avg_income'] = data['annualIncome'] / data['employmentLength']
    data['total_income'] = data['annualIncome'] * data['employmentLength']
    data['avg_loanAmnt'] = data['loanAmnt'] / data['term']
    data['mean_interestRate'] = data['interestRate'] / data['term']
    data['all_installment'] = data['installment'] * data['term']

    data['rest_money_rate'] = data['avg_loanAmnt'] / (data['annualIncome'] + 0.1)  # 287个收入为0
    data['rest_money'] = data['annualIncome'] - data['avg_loanAmnt']

    data['closeAcc'] = data['totalAcc'] - data['openAcc']
    data['ficoRange_mean'] = (data['ficoRangeHigh'] + data['ficoRangeLow']) / 2
    del data['ficoRangeHigh'], data['ficoRangeLow']

    data['rest_pubRec'] = data['pubRec'] - data['pubRecBankruptcies']

    data['rest_Revol'] = data['loanAmnt'] - data['revolBal']

    data['dis_time'] = data['issueDate_year'] - (2020 - data['earliesCreditLine_year'])
    for col in ['employmentTitle', 'grade', 'subGrade', 'regionCode', 'issueDate_month', 'postCode']:
        data['{}_count'.format(col)] = data.groupby([col])['id'].transform('count')

    return data


In [19]:


def plotroc(train_y, train_pred, test_y, val_pred):
    lw = 2
    ##train
    fpr, tpr, thresholds = roc_curve(train_y.values, train_pred, pos_label=1.0)
    train_auc_value = roc_auc_score(train_y.values, train_pred)
    ##valid
    fpr, tpr, thresholds = roc_curve(test_y.values, val_pred, pos_label=1.0)
    valid_auc_value = roc_auc_score(test_y.values, val_pred)

    return train_auc_value, valid_auc_value



In [20]:
#XGBoost
def xgb_model(train, target, test, k):
    # saveFeature_df = pd.read_csv('../feature/xgb_920_556_5_score.csv',header=None)
    #saveFeature_df2 = pd.read_csv('../feature/xgb_09-20_74_0.8_0.6_5_score.csv',header=None)
    
    # saveFeature_df.columns=['feature','score']
    #saveFeature_df2.columns=['feature','score']

    #saveFeature_list = list(saveFeature_df['feature'].values)
    #saveFeature_list2 = list(saveFeature_df2['feature'].values)
    
    # saveFeature_list = list(saveFeature_df[saveFeature_df['score']>10]['feature'])
    saveFeature_list=list(train.columns)
    feats = [f for f in saveFeature_list if f not in ['id', 'isDefault']]
    feaNum = len(feats)
    print('Current num of features:', len(feats))

    seeds = [2020,666666,188888]
    output_preds = 0
    xgb_oof_probs = np.zeros(train.shape[0])

    for seed in seeds:
        folds = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
        oof_probs = np.zeros(train.shape[0])

        offline_score = []
        feature_importance_df = pd.DataFrame()
        params = {'booster': 'gbtree',
                  'objective': 'binary:logistic',
                  'eval_metric': 'auc',
                  'min_child_weight': 5,
                  'max_depth': 8,
                  'subsample': ss,
                  'colsample_bytree': fs,
                  'eta': 0.01,

                  'seed': seed,
                  'nthread': -1,

                  'tree_method': 'gpu_hist'
                  }
        for i, (train_index, test_index) in enumerate(folds.split(train, target)):
            
            train_y, test_y = target[train_index], target[test_index]
            train_X, test_X = train[feats].iloc[train_index, :], train[feats].iloc[test_index, :]
            train_matrix = xgb.DMatrix(train_X, label=train_y, missing=np.nan)
            valid_matrix = xgb.DMatrix(test_X, label=test_y, missing=np.nan)
            test_matrix = xgb.DMatrix(test[feats], missing=np.nan)
            watchlist = [(train_matrix, 'train'), (valid_matrix, 'eval')]
            model = xgb.train(params, train_matrix, num_boost_round=100000, evals=watchlist, verbose_eval=100,
                              early_stopping_rounds=600)
            val_pred = model.predict(valid_matrix, ntree_limit=model.best_ntree_limit)
            train_pred = model.predict(train_matrix, ntree_limit=model.best_ntree_limit)
            xgb_oof_probs[test_index] += val_pred / len(seeds)
            # oof_probs[test_index] += val_pred
            test_pred = model.predict(test_matrix, ntree_limit=model.best_ntree_limit)

            # 绘制roc曲线
            train_auc_value, valid_auc_value = plotroc(train_y, train_pred, test_y, val_pred)
            print('train_auc:{},valid_auc{}'.format(train_auc_value, valid_auc_value))
            offline_score.append(valid_auc_value)
            print(offline_score)
            output_preds += test_pred / k / len(seeds)
            
#             sub_df = test[['id']].copy()
#             sub_df['isDefault'] = output_preds
#             off = test[['id']].copy()
#             subVal_df = train[['id']].copy()
#             subVal_df.loc[test_index,'isDefault'] = xgb_oof_probs[test_index]
#             outpath = '../user_data/fold/'
#             fold_score = round(valid_auc_value, 5)
#             sub_df.to_csv( outpath + str(fold_score) + '_' + str(feaNum) + '_' + nowtime + '_{}_{}_xgb.csv'.format(i, kflod_num),
#             index=False)
#             subVal_df.to_csv(outpath + str(fold_score) + '_' + str(feaNum) + '_' + nowtime + '_{}_{}_xgbVal.csv'.format(i, kflod_num),
#             index=False)

            fold_importance_df = pd.DataFrame()
            fold_importance_df["Feature"] = model.get_fscore().keys()
            fold_importance_df["importance"] = model.get_fscore().values()
            fold_importance_df["fold"] = i + 1

            feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
#             print(feature_importance_df.groupby(['Feature'])['importance'].mean().sort_values(ascending=False).head(50))
#             feature_sorted = feature_importance_df.groupby(['Feature'])['importance'].mean().sort_values(ascending=False)
#             feature_sorted.to_csv('../feature/xgb_{}_{}_{}_{}_score.csv'.format(i+1,nowtime, feaNum, kflod_num))

#             if i==3:
#                 break
#             gc.collect()


        print('all_auc:', roc_auc_score(target.values, oof_probs))
        print('OOF-MEAN-AUC:%.6f, OOF-STD-AUC:%.6f' % (np.mean(offline_score), np.std(offline_score)))
        feature_sorted = feature_importance_df.groupby(['Feature'])['importance'].mean().sort_values(ascending=False)
        feature_sorted.to_csv('xgb_importance.csv')
        top_features = feature_sorted.index
        print(feature_importance_df.groupby(['Feature'])['importance'].mean().sort_values(ascending=False).head(50))
    return output_preds, xgb_oof_probs, np.mean(offline_score), feaNum



In [21]:

if __name__ == '__main__':
    DATA_PATH = 'd:/桌面/学习资料/贷款违约预测/'
    print('读取数据...')
    data, train_label = data_preprocess(DATA_PATH=DATA_PATH)

    print('开始特征工程...')
    data = gen_basicFea(data)
    
    print('data.shape', data.shape)
    print('开始模型训练...')
    train = data[~data['isDefault'].isnull()].copy()
    target = train_label
    test = data[data['isDefault'].isnull()].copy()

    target_encode_cols = ['postCode', 'regionCode', 'homeOwnership', 'employmentTitle','title']

    kflod_num = 5
    ss = 0.8
    fs = 0.4

    class_list = ['postCode', 'regionCode', 'homeOwnership', 'employmentTitle','title']
    MeanEnocodeFeature = class_list  # 声明需要平均数编码的特征
    ME = MeanEncoder(MeanEnocodeFeature, target_type='classification')  # 声明平均数编码的类
    train = ME.fit_transform(train, target)  # 对训练数据集的X和y进行拟合
    # x_train_fav = ME.fit_transform(x_train,y_train_fav)#对训练数据集的X和y进行拟合
    test = ME.transform(test)  # 对测试集进行编码
    print('num0:mean_encode train.shape', train.shape, test.shape)

    train, test = kfold_stats_feature(train, test, target_encode_cols, kflod_num)
    print('num1:target_encode train.shape', train.shape, test.shape)
    ### target encoding目标编码，回归场景相对来说做目标编码的选择更多，不仅可以做均值编码，还可以做标准差编码、中位数编码等
    enc_cols = []
    stats_default_dict = {
        'max': train['isDefault'].max(),
        'min': train['isDefault'].min(),
        'median': train['isDefault'].median(),
        'mean': train['isDefault'].mean(),
        'sum': train['isDefault'].sum(),
        'std': train['isDefault'].std(),
        'skew': train['isDefault'].skew(),
        'kurt': train['isDefault'].kurt(),
        'mad': train['isDefault'].mad()
    }
    ### 暂且选择这三种编码
    enc_stats = ['max', 'min', 'skew', 'std']
    skf = KFold(n_splits=kflod_num, shuffle=True, random_state=6666)
    for f in tqdm(['postCode', 'regionCode', 'homeOwnership', 'employmentTitle','title']):
        enc_dict = {}
        for stat in enc_stats:
            enc_dict['{}_target_{}'.format(f, stat)] = stat
            train['{}_target_{}'.format(f, stat)] = 0
            test['{}_target_{}'.format(f, stat)] = 0
            enc_cols.append('{}_target_{}'.format(f, stat))
        for i, (trn_idx, val_idx) in enumerate(skf.split(train, target)):
            trn_x, val_x = train.iloc[trn_idx].reset_index(drop=True), train.iloc[val_idx].reset_index(drop=True)
            enc_df = trn_x.groupby(f, as_index=False)['isDefault'].agg(enc_dict)
            val_x = val_x[[f]].merge(enc_df, on=f, how='left')
            test_x = test[[f]].merge(enc_df, on=f, how='left')
            for stat in enc_stats:
                val_x['{}_target_{}'.format(f, stat)] = val_x['{}_target_{}'.format(f, stat)].fillna(
                    stats_default_dict[stat])
                test_x['{}_target_{}'.format(f, stat)] = test_x['{}_target_{}'.format(f, stat)].fillna(
                    stats_default_dict[stat])
                train.loc[val_idx, '{}_target_{}'.format(f, stat)] = val_x['{}_target_{}'.format(f, stat)].values
                test['{}_target_{}'.format(f, stat)] += test_x['{}_target_{}'.format(f, stat)].values / skf.n_splits

    print('num2:target_encode train.shape', train.shape, test.shape)

    train.drop(['postCode', 'regionCode', 'homeOwnership', 'employmentTitle','title'], axis=1, inplace=True)
    test.drop(['postCode', 'regionCode', 'homeOwnership', 'employmentTitle','title'], axis=1, inplace=True)
    print('输入数据维度：', train.shape, test.shape)
    
    xgb_preds, xgb_oof, xgb_score, feaNum = xgb_model(train=train, target=target, test=test, k=kflod_num)

    lgb_score = round(xgb_score, 5)
    sub_df = test[['id']].copy()
    sub_df['isDefault'] = xgb_preds
    off = test[['id']].copy()
    subVal_df = train[['id']].copy()
    subVal_df['isDefault'] = xgb_oof
    outpath = '../user_data/'

    all_auc_score = roc_auc_score(train_label, subVal_df['isDefault'])
    print('整体指标得分：', all_auc_score)
    all_auc_score = round(all_auc_score, 5)

    sub_df.to_csv('xgb1.csv',index=False)
    subVal_df.to_csv('xgb1Val.csv',index=False)
    # sub_df.to_csv(
    #     outpath + str(all_auc_score) + '_' + str(feaNum) + '_' + nowtime + '_{}_{}_{}_xgb.csv'.format(ss, fs,
    #                                                                                                   kflod_num),
    #     index=False)
    # subVal_df.to_csv(
    #     outpath + str(all_auc_score) + '_' + str(feaNum) + '_' + nowtime + '_{}_{}_{}_subVal.csv'.format(ss, fs,
    #                                                                                                      kflod_num),
    #     index=False)

读取数据...
train.shape (800000, 47)
test.shape (200000, 46)
初始拼接后： (1000000, 47)
n特征处理后： (1000000, 53)
count编码后： (1000000, 61)
1data.shape (1000000, 53)
2_data.shape (1000000, 57)
预处理完毕 (1000000, 58)
开始特征工程...
data.shape (1000000, 74)
开始模型训练...
num0:mean_encode train.shape (800000, 84) (200000, 84)
num1:target_encode train.shape (800000, 89) (200000, 89)


100%|██████████| 5/5 [05:10<00:00, 62.12s/it] 


num2:target_encode train.shape (800000, 109) (200000, 109)
输入数据维度： (800000, 104) (200000, 104)
Current num of features: 102
[0]	train-auc:0.71510	eval-auc:0.70796
[100]	train-auc:0.73560	eval-auc:0.72415
[200]	train-auc:0.74143	eval-auc:0.72737
[300]	train-auc:0.74712	eval-auc:0.73013


: 

In [2]:
def xgb_model2(train, target, test, k):
    saveFeature_df = pd.read_csv('xgb_importance.csv',header=None)
#     saveFeature_df2 = pd.read_csv('xgb_09-20_74_0.8_0.6_5_score.csv',header=None)
    
    saveFeature_df.columns=['feature','score']
#     saveFeature_df2.columns=['feature','score']

    saveFeature_list = list(saveFeature_df['feature'].values)
#     saveFeature_list2 = list(saveFeature_df2['feature'].values)
    
    saveFeature_list = list(saveFeature_df[saveFeature_df['score']>0]['feature'])
    saveFeature_list=list(train.columns)
    feats = [f for f in saveFeature_list if f not in ['id', 'isDefault']]
    feaNum = len(feats)
    print('Current num of features:', len(feats))

    seeds = [2020,666666,188888]
    output_preds = 0
    xgb_oof_probs = np.zeros(train.shape[0])

    for seed in seeds:
        folds = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
        oof_probs = np.zeros(train.shape[0])

        offline_score = []
        feature_importance_df = pd.DataFrame()
        params = {'booster': 'gbtree',
                  'objective': 'binary:logistic',
                  'eval_metric': 'auc',
                  'min_child_weight': 5,
                  'max_depth': 8,
                  'subsample': ss,
                  'colsample_bytree': fs,
                  'eta': 0.01,

                  'seed': seed,
                  'nthread': -1,

                  'tree_method': 'gpu_hist'
                  }
        for i, (train_index, test_index) in enumerate(folds.split(train, target)):
            
            train_y, test_y = target[train_index], target[test_index]
            train_X, test_X = train[feats].iloc[train_index, :], train[feats].iloc[test_index, :]
            train_matrix = xgb.DMatrix(train_X, label=train_y, missing=np.nan)
            valid_matrix = xgb.DMatrix(test_X, label=test_y, missing=np.nan)
            test_matrix = xgb.DMatrix(test[feats], missing=np.nan)
            watchlist = [(train_matrix, 'train'), (valid_matrix, 'eval')]
            model = xgb.train(params, train_matrix, num_boost_round=100000, evals=watchlist, verbose_eval=100,
                              early_stopping_rounds=600)
            val_pred = model.predict(valid_matrix, ntree_limit=model.best_ntree_limit)
            train_pred = model.predict(train_matrix, ntree_limit=model.best_ntree_limit)
            xgb_oof_probs[test_index] += val_pred / len(seeds)
            # oof_probs[test_index] += val_pred
            test_pred = model.predict(test_matrix, ntree_limit=model.best_ntree_limit)

            # 绘制roc曲线
            train_auc_value, valid_auc_value = plotroc(train_y, train_pred, test_y, val_pred)
            print('train_auc:{},valid_auc{}'.format(train_auc_value, valid_auc_value))
            offline_score.append(valid_auc_value)
            print(offline_score)
            output_preds += test_pred / k / len(seeds)
            
#             sub_df = test[['id']].copy()
#             sub_df['isDefault'] = output_preds
#             off = test[['id']].copy()
#             subVal_df = train[['id']].copy()
#             subVal_df.loc[test_index,'isDefault'] = xgb_oof_probs[test_index]
#             outpath = '../user_data/fold/'
#             fold_score = round(valid_auc_value, 5)
#             sub_df.to_csv( outpath + str(fold_score) + '_' + str(feaNum) + '_' + nowtime + '_{}_{}_xgb.csv'.format(i, kflod_num),
#             index=False)
#             subVal_df.to_csv(outpath + str(fold_score) + '_' + str(feaNum) + '_' + nowtime + '_{}_{}_xgbVal.csv'.format(i, kflod_num),
#             index=False)

            fold_importance_df = pd.DataFrame()
            fold_importance_df["Feature"] = model.get_fscore().keys()
            fold_importance_df["importance"] = model.get_fscore().values()
            fold_importance_df["fold"] = i + 1

            feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
#             print(feature_importance_df.groupby(['Feature'])['importance'].mean().sort_values(ascending=False).head(50))
#             feature_sorted = feature_importance_df.groupby(['Feature'])['importance'].mean().sort_values(ascending=False)
#             feature_sorted.to_csv('../feature/xgb_{}_{}_{}_{}_score.csv'.format(i+1,nowtime, feaNum, kflod_num))

#             if i==3:
#                 break
#             gc.collect()


        print('all_auc:', roc_auc_score(target.values, oof_probs))
        print('OOF-MEAN-AUC:%.6f, OOF-STD-AUC:%.6f' % (np.mean(offline_score), np.std(offline_score)))
        feature_sorted = feature_importance_df.groupby(['Feature'])['importance'].mean().sort_values(ascending=False)
#         feature_sorted.to_csv('../feature/xgb_importance.csv')
        top_features = feature_sorted.index
        print(feature_importance_df.groupby(['Feature'])['importance'].mean().sort_values(ascending=False).head(50))
    return output_preds, xgb_oof_probs, np.mean(offline_score), feaNum




In [21]:

del train,test
gc.collect()
print('data.shape', data.shape)
print('开始模型训练...')
train = data[~data['isDefault'].isnull()].copy()
target = train_label
test = data[data['isDefault'].isnull()].copy()

target_encode_cols = ['postCode', 'regionCode', 'homeOwnership', 'employmentTitle','title']

kflod_num = 5
ss = 0.8
fs = 0.4

class_list = ['postCode', 'regionCode', 'homeOwnership', 'employmentTitle','title']
MeanEnocodeFeature = class_list  # 声明需要平均数编码的特征
ME = MeanEncoder(MeanEnocodeFeature, target_type='classification')  # 声明平均数编码的类
train = ME.fit_transform(train, target)  # 对训练数据集的X和y进行拟合
# x_train_fav = ME.fit_transform(x_train,y_train_fav)#对训练数据集的X和y进行拟合
test = ME.transform(test)  # 对测试集进行编码
print('num0:mean_encode train.shape', train.shape, test.shape)

train, test = kfold_stats_feature(train, test, target_encode_cols, kflod_num)
print('num1:target_encode train.shape', train.shape, test.shape)
### target encoding目标编码，回归场景相对来说做目标编码的选择更多，不仅可以做均值编码，还可以做标准差编码、中位数编码等
enc_cols = []
stats_default_dict = {
    'max': train['isDefault'].max(),
    'min': train['isDefault'].min(),
    'median': train['isDefault'].median(),
    'mean': train['isDefault'].mean(),
    'sum': train['isDefault'].sum(),
    'std': train['isDefault'].std(),
    'skew': train['isDefault'].skew(),
    'kurt': train['isDefault'].kurt(),
    'mad': train['isDefault'].mad()
}
### 暂且选择这三种编码
enc_stats = ['max', 'min', 'skew', 'std']
skf = KFold(n_splits=kflod_num, shuffle=True, random_state=6666)
for f in tqdm(['postCode', 'regionCode', 'homeOwnership', 'employmentTitle','title']):
    enc_dict = {}
    for stat in enc_stats:
        enc_dict['{}_target_{}'.format(f, stat)] = stat
        train['{}_target_{}'.format(f, stat)] = 0
        test['{}_target_{}'.format(f, stat)] = 0
        enc_cols.append('{}_target_{}'.format(f, stat))
    for i, (trn_idx, val_idx) in enumerate(skf.split(train, target)):
        trn_x, val_x = train.iloc[trn_idx].reset_index(drop=True), train.iloc[val_idx].reset_index(drop=True)
        enc_df = trn_x.groupby(f, as_index=False)['isDefault'].agg(enc_dict)
        val_x = val_x[[f]].merge(enc_df, on=f, how='left')
        test_x = test[[f]].merge(enc_df, on=f, how='left')
        for stat in enc_stats:
            val_x['{}_target_{}'.format(f, stat)] = val_x['{}_target_{}'.format(f, stat)].fillna(
                stats_default_dict[stat])
            test_x['{}_target_{}'.format(f, stat)] = test_x['{}_target_{}'.format(f, stat)].fillna(
                stats_default_dict[stat])
            train.loc[val_idx, '{}_target_{}'.format(f, stat)] = val_x['{}_target_{}'.format(f, stat)].values
            test['{}_target_{}'.format(f, stat)] += test_x['{}_target_{}'.format(f, stat)].values / skf.n_splits

print('num2:target_encode train.shape', train.shape, test.shape)

train.drop(['postCode', 'regionCode', 'homeOwnership', 'employmentTitle','title'], axis=1, inplace=True)
test.drop(['postCode', 'regionCode', 'homeOwnership', 'employmentTitle','title'], axis=1, inplace=True)
print('输入数据维度：', train.shape, test.shape)


data.shape (1000000, 74)
开始模型训练...
num0:mean_encode train.shape (800000, 84) (200000, 84)
num1:target_encode train.shape (800000, 89) (200000, 89)


 60%|██████    | 3/5 [00:18<00:12,  6.04s/it]

: 

In [1]:

xgb_preds, xgb_oof, xgb_score, feaNum = xgb_model2(train=train, target=target, test=test, k=kflod_num)

lgb_score = round(xgb_score, 5)
sub_df = test[['id']].copy()
sub_df['isDefault'] = xgb_preds
off = test[['id']].copy()
subVal_df = train[['id']].copy()
subVal_df['isDefault'] = xgb_oof
outpath = '../user_data/'

all_auc_score = roc_auc_score(train_label, subVal_df['isDefault'])
print('整体指标得分：', all_auc_score)
all_auc_score = round(all_auc_score, 5)

sub_df.to_csv(outpath+'xgb1.csv',index=False)
subVal_df.to_csv(outpath+'xgb1Val.csv',index=False)

NameError: name 'xgb_model2' is not defined